### Importing Datasets 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import spacy
import spacy
nlp = spacy.load('en_core_web_sm')

In [3]:
# Read yelp data
df_yelp = pd.read_csv('data/yelp_labelled.txt', sep='\t',header=None)

In [4]:
df_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
# Let's assign some relavant column names
columan_name = ['Review', 'Sentiment']
df_yelp.columns = columan_name

In [6]:
df_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
df_yelp.shape

(1000, 2)

In [9]:
# Read amazon data
df_amazon = pd.read_csv('data/amazon_cells_labelled.txt',sep='\t',header=None)
df_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [11]:
# Update the column names
df_amazon.columns = columan_name
df_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [12]:
df_amazon.shape

(1000, 2)

In [13]:
# Read IMDB data
df_imdb = pd.read_csv('data/imdb_labelled.txt',sep='\t',header=None)
df_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [14]:
df_imdb.columns = columan_name
df_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [15]:
df_imdb.shape

(748, 2)

### Data processing 

In [16]:
# Combining all data into a single dataframe
df = df_yelp.append([df_amazon, df_imdb],ignore_index=True)

In [17]:
df.shape

(2748, 2)

In [18]:
df.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [19]:
# Distributuion of sentiments
df['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [20]:
# 1386 positive sentiments
# 1362 negative sentiments

In [21]:
# Check for null values
df.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [23]:
x = df['Review']
y = df['Sentiment']

In [24]:
# Get punctuations
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [26]:
# Get stopwords
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
stopwords

['among',
 'whence',
 'also',
 'twelve',
 'across',
 'seems',
 'of',
 'fifty',
 're',
 'therefore',
 'somehow',
 'will',
 'wherein',
 'must',
 'hundred',
 'regarding',
 '‘d',
 'below',
 'nevertheless',
 'always',
 '‘ve',
 'indeed',
 'whose',
 'part',
 'along',
 'once',
 'formerly',
 'forty',
 'even',
 'off',
 'under',
 'us',
 'too',
 'myself',
 'moreover',
 'becoming',
 'somewhere',
 'first',
 'latter',
 'themselves',
 'third',
 'using',
 'last',
 'how',
 'thru',
 'most',
 'as',
 'get',
 'becomes',
 'became',
 'noone',
 'mostly',
 'your',
 'call',
 'towards',
 '‘ll',
 'three',
 'make',
 'so',
 'hereby',
 'otherwise',
 'are',
 'still',
 'twenty',
 'himself',
 'thereafter',
 'itself',
 'neither',
 'nor',
 'often',
 'do',
 'yet',
 'rather',
 'everyone',
 'the',
 'which',
 'her',
 'take',
 'very',
 'on',
 'next',
 'can',
 'ourselves',
 'keep',
 'yours',
 'afterwards',
 'into',
 'full',
 'thus',
 'any',
 'these',
 'n’t',
 'everywhere',
 'my',
 'yourselves',
 'seemed',
 'less',
 'some',
 'ha

#### Data cleanup 

In [27]:
# Data cleanup function
# If root form of that word is not pronoun then convert that into lower form
# If that word is a proper noun, then directly take lower form, because there is no lemma for proper noun
def text_data_cleaning(sentence):
    tokens = []
    
    doc = nlp(sentence)
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
 
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [28]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

### Feature Engineering using TF-IDF 

In [29]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [30]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)

In [31]:
classifier = LinearSVC()

### Training The Model 

In [32]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [33]:
x_train.shape, x_test.shape

((2198,), (550,))

In [34]:
x_train

2572    An Italian reviewer called this "a small, grea...
526                          And it was way to expensive.
1509    As an earlier review noted, plug in this charg...
144     Nice blanket of moz over top but i feel like t...
2483    The film gives meaning to the phrase, "Never i...
                              ...                        
763                       But the service was beyond bad.
835     I paid the bill but did not tip because I felt...
1653              Utter crap.. Sound quality is TERRIBLE.
2607    The only place good for this film is in the ga...
2732             She is as lovely as usual, this cutie!  
Name: Review, Length: 2198, dtype: object

In [35]:
# Fit the train
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])

In [36]:
clf.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x7fea2be22440>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept

### Prediction 

In [37]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [38]:
y_pred = clf.predict(x_test)

In [39]:
# Confusion Matrix
confusion_matrix(y_test, y_pred)

array([[204,  75],
       [ 49, 222]])

In [40]:
# Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.73      0.77       279
           1       0.75      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.78      0.78      0.77       550
weighted avg       0.78      0.77      0.77       550



In [41]:
accuracy_score(y_test, y_pred)

0.7745454545454545

In [42]:
clf.predict(["It's hard to live here!"])

array([0])

In [43]:
clf.predict(["It tastes amazing", "The phone is rubbish"])

array([1, 0])